Here I want to check wheter the change of price which should occur based on the change of the treasury yield actually ocurred for the ETF and Futures. For instance, if the yield on the treasury note fell between Q1 and Q2, I would expect the price of the Futures contract and ETF to rise, which is an implicit assumption for the trading strategy to work. Here I want to check wheter this relationship actually holds.

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
#Two Year Treasury Yield
relative_path = os.path.join('..', '..',"Data",'Consolidated',"TreasuryYield", "TUQE.csv")
TUY=pd.read_csv(relative_path,index_col=0)
#Ten Year Treasury Yield
relative_path = os.path.join('..', '..',"Data",'Consolidated',"TreasuryYield", "TYQE.csv")
TYY=pd.read_csv(relative_path,index_col=0)
#Long-Term IEF ETF
relative_path = os.path.join('..', '..',"Data",'Consolidated',"ETF", "IEF.csv")
IEF=pd.read_csv(relative_path,index_col=0)
#Short-Term SHY ETF
relative_path = os.path.join('..', '..',"Data",'Consolidated',"ETF", "SHY.csv")
SHY=pd.read_csv(relative_path,index_col=0)
#TYc1
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TYc1ME.csv")
TYc1=pd.read_csv(relative_path,index_col=0)
#TYc2
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TYc2ME.csv")
TYc2=pd.read_csv(relative_path,index_col=0)
#TYc3
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TYc3ME.csv")
TYc3=pd.read_csv(relative_path,index_col=0)
#TUc1
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TUc1ME.csv")
TUc1=pd.read_csv(relative_path,index_col=0)
#TUc2
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TUc2ME.csv")
TUc2=pd.read_csv(relative_path,index_col=0)
#TUc3
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TUc3ME.csv")
TUc3=pd.read_csv(relative_path,index_col=0)

# ETFs

In [3]:
#merge interest rate data and etf data
TYETF=TYY.merge(IEF,how="inner",left_index=True, right_index=True)
#drop columns that are not needed
TYETF.drop(["Bid","Ask"], axis=1,inplace=True)
#Create columns for sign change
TYETF["Change10Y"]=np.nan
TYETF["ChangeETF"]=np.nan

#do the same thing for SHY ETF and two year yield

#merge interest rate data and etf data
TUETF=TUY.merge(SHY,how="inner",left_index=True, right_index=True)
#drop columns that are not needed
TUETF.drop(["Bid","Ask"], axis=1,inplace=True)
#Create columns for sign change
TUETF["Change2Y"]=np.nan
TUETF["ChangeETF"]=np.nan

In [4]:
#function that assigns sign change of interest rate
def signassignor(value):
    if value>0:
        return (1)
    elif value<0:
        return (-1)
    else:
        return(0)

In [5]:
#check wheter value changed as predicted
i=0
for index, row in TYETF.iterrows():
    i=i+1
    if i==1:
        index_1=index
    elif i>1:
        #check how interest rate value changed
        ichange=TYETF.loc[index,"10Y"]-TYETF.loc[index_1,"10Y"]
        #assign value according to interest rate change
        TYETF.loc[index,"Change10Y"]=signassignor(ichange)
        #check how etf value changed
        ETFchange=TYETF.loc[index,"Close"]-TYETF.loc[index_1,"Close"]
        #assigne value according to change
        TYETF.loc[index,"ChangeETF"]=signassignor(ETFchange)
        #update previous index value
        index_1=index
#If both changes equal 0, than price change was correct
TYETF["ChangeBoth"]=TYETF["Change10Y"]+TYETF["ChangeETF"]
#Count how often each value appeared
ETF10Y=pd.value_counts(TYETF["ChangeBoth"])

In [6]:
#check wheter value changed as predicted
i=0
for index, row in TUETF.iterrows():
    i=i+1
    if i==1:
        index_1=index
    elif i>1:
        #check how interest rate value changed
        ichange=TUETF.loc[index,"2Y"]-TUETF.loc[index_1,"2Y"]
        #assign value according to interest rate change
        TUETF.loc[index,"Change2Y"]=signassignor(ichange)
        #check how etf value changed
        ETFchange=TUETF.loc[index,"Close"]-TUETF.loc[index_1,"Close"]
        #assigne value according to change
        TUETF.loc[index,"ChangeETF"]=signassignor(ETFchange)
        #update previous index value
        index_1=index
#If both changes equal 0, than price change was correct
TUETF["ChangeBoth"]=TUETF["Change2Y"]+TUETF["ChangeETF"]
#Count how often each value appeared
ETF2Y=pd.value_counts(TUETF["ChangeBoth"])

In [ ]:
ETF2Y

In [ ]:
ETF10Y

# Futures Contracts

In [9]:
TUY.index=pd.to_datetime(TUY.index)
TYY.index=pd.to_datetime(TYY.index)

In [10]:
def value_counter(yield_df,future_df):
    future_df.index=pd.to_datetime(future_df.index)
    future_df=future_df["Close"].resample("Q").last()
    future_df=future_df.iloc[1:]
    yieldchange=np.sign(yield_df.diff())
    futurechange=np.sign(future_df.diff())
    change=futurechange+yieldchange
    change_df=change.dropna().value_counts()
    return(change_df)


In [11]:
def accuracy(value_df):
    return(value_df[0]/sum(value_df))

In [12]:
value_dic={
    "ETF: SHY":accuracy(ETF2Y),
    "ETF: IEF":accuracy(ETF10Y),
    "Future: TUc1":accuracy(value_counter(TUY["2Y"],TUc1)),
    "Future: TUc2":accuracy(value_counter(TUY["2Y"],TUc2)),
    "Future: TUc3":accuracy(value_counter(TUY["2Y"],TUc3)),
    "Future: TYc1":accuracy(value_counter(TYY["10Y"],TYc1)),
    "Future: TYc2":accuracy(value_counter(TYY["10Y"],TYc2)),
    "Future: TYc3":accuracy(value_counter(TYY["10Y"],TYc3))
}

ERAR=pd.DataFrame(value_dic,index=["Expected Relationship Accuracy Ratio"]).T

In [13]:
relative_path = os.path.join('..', '..',"Data","Results","Descriptive","ERAR.xlsx")
ERAR.to_excel(relative_path)